In [1]:
# !pip install -U nltk
# !pip install easy-rouge

In [2]:
import pandas as pd
import string, re
import numpy as np
from tqdm import tqdm
from rouge.rouge import rouge_n_sentence_level
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/dojo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
df_bert = pd.read_csv('data/results_BERT.csv')

In [4]:
df_t5 = pd.read_csv('data/results_T5.csv')

In [5]:
df = pd.concat([df_bert, df_t5], axis=1).dropna().reset_index(drop=True)

In [6]:
df.head()

,Truths (Bert),Predicted (Bert),Truths (T5),Predicted (T5)
0,This Arthurian hero stars in an alliterative M...,sir knight sir sir sir sir knight knight knigh...,This Arthurian hero stars in an alliterative M...,"Who wrote the book ""The Crown""?"
1,"In 2001 she made history: her album ""J. Lo"" & ...","she 1997 her her her her her her her "" "" "" "" ""...","In 2001 she made history: her album ""J. Lo"" & ...",When did Lopez release her second studio album...
2,Among the 854 people the Russian Orthodox Chur...,in 5 this heads this this thisev alexei alexei...,Among the 854 people the Russian Orthodox Chur...,In what year did Nicholas Alexei die?
3,"Joe Cocker sang ""Up Where We Belong"", the love...","in "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" ""...","Joe Cocker sang ""Up Where We Belong"", the love...","Who wrote ""Up Where We Belong"" for the 1982 fi..."
4,"This 1917 British declaration favored ""the est...","in 1930,,,, the the the to to to to to to to t...","This 1917 British declaration favored ""the est...",When was the Balfour Declaration issued?


In [7]:
df_out = pd.DataFrame({
    'Rouge (T5)': np.zeros(len(df)),
    'Rouge (Bert)': np.zeros(len(df)),
    'BLEU (T5)': np.zeros(len(df)),
    'BLEU (Bert)': np.zeros(len(df)),
    'METEOR (T5)': np.zeros(len(df)),
    'METEOR (Bert)': np.zeros(len(df)),
})

In [8]:
def clean_sentence(collection):
    collection = collection.translate(str.maketrans('','',string.punctuation))
    collection = re.sub(r'\d+', '', collection)
    collection = collection.strip()
    return collection.split()

In [9]:
for i in tqdm(range(len(df))):
    row = df.iloc[i]
    truth = row['Truths (T5)'].lower()
    t5 = row['Predicted (T5)'].lower()
    bert = row['Predicted (Bert)'].lower()

    recall, precision, rouge = rouge_n_sentence_level(
        truth.split(),
        t5.split(),
        2)
    df_out.iloc[i]['Rouge (T5)'] = rouge
    
    recall, precision, rouge = rouge_n_sentence_level(
        truth.split(),
        bert.split(),
        2)
    df_out.iloc[i]['Rouge (Bert)'] = rouge

    df_out.iloc[i]['BLEU (Bert)'] = nltk.translate.bleu_score.sentence_bleu(
        [truth], bert)
    df_out.iloc[i]['BLEU (T5)'] = nltk.translate.bleu_score.sentence_bleu(
        [truth], t5)

    df_out.iloc[i]['METEOR (Bert)'] = nltk.translate.meteor_score.meteor_score(truth, bert)
    df_out.iloc[i]['METEOR (T5)'] = nltk.translate.meteor_score.meteor_score(truth, t5)

 12%|█▏        | 2/17 [00:01<00:11,  1.32it/s]/home/dojo/dev/.pyvenvs/qa-old/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
 18%|█▊        | 3/17 [00:01<00:06,  2.17it/s]/home/dojo/dev/.pyvenvs/qa-old/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
100%|██████████| 17/17 [00:03<00:00,  5.22it/s]


In [11]:
final_results = pd.concat([df, df_out], axis=1)#.to_csv('data/results_with_metrics.csv')

In [13]:
final_results.head()

,Truths (Bert),Predicted (Bert),Truths (T5),Predicted (T5),Rouge (T5),Rouge (Bert),BLEU (T5),BLEU (Bert),METEOR (T5),METEOR (Bert)
0,This Arthurian hero stars in an alliterative M...,sir knight sir sir sir sir knight knight knigh...,This Arthurian hero stars in an alliterative M...,"Who wrote the book ""The Crown""?",0.0000,0.0,2.956733e-02,1.065140e-01,0.000000,0.000000
1,"In 2001 she made history: her album ""J. Lo"" & ...","she 1997 her her her her her her her "" "" "" "" ""...","In 2001 she made history: her album ""J. Lo"" & ...",When did Lopez release her second studio album...,0.0625,0.0,2.945475e-01,1.149156e-01,0.250000,0.147059
2,Among the 854 people the Russian Orthodox Chur...,in 5 this heads this this thisev alexei alexei...,Among the 854 people the Russian Orthodox Chur...,In what year did Nicholas Alexei die?,0.0000,0.0,5.670795e-79,9.853621e-02,0.000000,0.217391
3,"Joe Cocker sang ""Up Where We Belong"", the love...","in "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" "" ""...","Joe Cocker sang ""Up Where We Belong"", the love...","Who wrote ""Up Where We Belong"" for the 1982 fi...",0.2500,0.0,5.060635e-01,2.270750e-155,0.208333,0.142857
4,"This 1917 British declaration favored ""the est...","in 1930,,,, the the the to to to to to to to t...","This 1917 British declaration favored ""the est...",When was the Balfour Declaration issued?,0.0000,0.0,9.077182e-02,7.196935e-02,0.000000,0.000000
